In [ ]:
# REQUIRED INPUT
# data.norm.sub_phyloseqObject.RData: phyloseq object including 
# Perturbations variable in sample_data

In [ ]:
###USER-INTERACTION###
# Cells with this header require user interaction, or adaptation 
# of the code to the specific case study.

In [ ]:
# Load packages and functions
source("robust.clustering.metagenomics.functions.r")

In [ ]:
###USER-INTERACTION###
# Define suffix for the current case study files
labelExp <- "CommunityExample" # For example: BifFae
setwd(labelExp)

In [ ]:
###USER-INTERACTION###
# Define variables name from sampling table of phyloseq object
# Sample attribute for time points
stepVar <- "time"
# Sample attribute for subject 
subjectVar <- "subject"

In [ ]:
# Run robust clustering for up to 10 clusters
dir.create('RobustClustering')
file.copy('data.norm.sub_phyloseqObject.RData','RobustClustering/',copy.date=TRUE,overwrite=TRUE)
robust.clustering.all.steps('RobustClustering','data.norm.sub_phyloseqObject.RData',labelExp,stepVar,maxClus=10)

In [ ]:
# Print state time serie, per subject
setwd(paste('RobustClustering/',labelExp,'_all/',sep=''))
fout <- paste('data.normAndDist_definitiveClustering_',labelExp,'.RData',sep='')
file.copy(fout,'../..',copy.date=TRUE,overwrite=TRUE)
load(fout)
table(sample_data(data.norm)$cluster)
tableSerie <- stateSerieTable(data.norm, stepVar, subjectId = subjectVar)
timeSerieHeatMap(tableSerie, "./", paste('statesSequence_all',labelExp,".pdf",sep=''))
setwd('../..')

In [ ]:
###USER-INTERACTION###
# To compute the cluster, when you decide a fix number of them,
# to adapt better to the case study characteristics.
# Example with k=3
k=3
dir.create(paste('RobustClustering_k',k,sep=''))
file.copy('data.norm.sub_phyloseqObject.RData',paste('RobustClustering_k',k,sep=''),copy.date=TRUE,overwrite=TRUE)
robust.clustering.all.steps(paste('RobustClustering_k',k,sep=''),'data.norm.sub_phyloseqObject.RData',labelExp,'time',minClus=k,maxClus=k)
setwd(paste('RobustClustering_k',k,'/',labelExp,'_all/',sep=''))
fout <- paste('data.normAndDist_definitiveClustering_',labelExp,'.RData',sep='')
file.copy(fout,'../..',copy.date=TRUE,overwrite=TRUE)
load(fout)
table(sample_data(data.norm)$cluster)
tableSerie <- stateSerieTable(data.norm, stepVar, subjectId = subjectVar)
timeSerieHeatMap(tableSerie, "./", paste('statesSequence_all',labelExp,".pdf",sep=''))
setwd('../..')

In [ ]:
# Starting MDPbiome
dir.create('MDPbiome')
file.copy('../MDPbiome_template/','.',copy.date=TRUE,recursive=TRUE,overwrite=TRUE)
file.rename('MDPbiome_template','MDPbiome')
file.copy(paste('data.normAndDist_definitiveClustering_',labelExp,'.RData',sep=''),'MDPbiome/Data/',copy.date=TRUE,overwrite=TRUE)

In [ ]:
# Load MDPbiome sources
setwd('MDPbiome/Src/')
source("initMDPBiome.R")
dirdata <- "../Data/"
setwd(dirdata)

In [ ]:
# Read OTU and mapping table
phyloObject_datafile <- paste('data.normAndDist_definitiveClustering_',labelExp,'.RData', sep = "")
load(phyloObject_datafile)

In [ ]:
###USER-INTERACTION###
# Rename clusters for clarity
levels(sample_data(data.norm)$cluster) <- c("dysbiosis","risky","healthy")

In [ ]:
# Rename perturbations field for clarity
sample_data(data.norm)$pert <- as.character(sample_data(data.norm)$Perturbations)
Perturbations <- c('pert') # Only 1 perturbation, with different values

In [ ]:
# Associate perturbation to the sample it results in, rather than to 
# the state where it is applied. To be the same than in microbiome
# sampling, where the mapping value of the perturbations is associated
# to the sample after the perturbation was applied.
for(subject in unique(get_variable(data.norm,subjectVar))){
  subject.data <- phyloSubset(data.norm, subjectVar, subject)
  vectPert <- NULL
  vectPert <- get_variable(subject.data,'pert')
  newVecPert <- c('NA',vectPert[1:(length(vectPert)-1)])
  sample_data(data.norm)[sample_names(subject.data), "pert"] <- newVecPert
} # end-for move perturbation

In [ ]:
# Generic functions that could be useful for compute Utility Function
# Mainly useful in MDPbiomeGEM (simulated data with GEM)
concMetabolite <- function(phyloObj,subjectVar, subject, met){
  subject.data <- phyloSubset(phyloObj, subjectVar, subject)
  # concentrations <- get_variable(subject.data,met) # vector
  concentrations <- sample_data(subject.data)[,c(met)] # sample_data structure
  return(concentrations)
}
# Compute mean of utility variable in all samples of a given cluster
clusterUtilityFunction <- function(clusterId){
  cluster.data <- phyloSubset(data.norm, "cluster", clusterId)
  scores <- get_variable(cluster.data,goalVar)
  return(mean(scores))
}

In [ ]:
###USER-INTERACTION###  
# To define name utility function
# Ex: To maximize concentration of butyrate
goalVar <- "SCFAincrease"
# To compute utility function
# Each sample must have a <goalVar> variable in the sample_data phyloseq
# object (that clusterUtilityFunction() will use). It could be fill in
# following the next example.
# Example:
metName <- 'Butyrate_C4H8O2'
subjects <- unique(get_variable(data.norm,'subject'))
for (isubject in subjects){
  vecConc <- concMetabolite(data.norm,subjectVar,isubject,metName)
  newVec <- as.numeric(rep(0,nsamples(vecConc)))
  for(pos in 2:(nsamples(vecConc))){
    newVec[pos] <- get_variable(vecConc[pos],metName) - get_variable(vecConc[pos-1],metName)
  } # end-for pos
  sample_data(data.norm)[sample_names(vecConc), goalVar] <- newVec
} # end-for isubject

In [ ]:
print('>> Utility Function by state:')
# Print computed values about change in utility function;
# here, a metabolite concentration
for (state in levels(sample_data(data.norm)$cluster)){
  ss=subset_samples(subset_samples(data.norm,(cluster==state)),pert!='NA')
  avg=mean(get_variable(ss,goalVar))
  print(paste(goalVar,state,':',avg))
} # end-for state

In [ ]:
print('>> Utility Function by perturbation:')
sample_data(data.norm)$pert=as.factor(sample_data(data.norm)$pert)
for (p in levels(sample_data(data.norm)$pert)){
  ss=subset_samples(data.norm,(pert==p))
  avg=mean(get_variable(ss,goalVar))
  print(paste(p,'(',nsamples(ss), 'samples)',':',avg))
} # end-for perturbation

In [ ]:
###USER-INTERACTION###
# To assign cluster preferences if expert knowledge available
# cluster_preference <- c(0,0,1)

In [ ]:
# Compute vector of states (i.e. cluster) preferences
states <- levels(sample_data(data.norm)$cluster)
goal_preference <- sapply(states, clusterUtilityFunction)
cluster_preference <- goal_preference

In [ ]:
save(data.norm,goal_preference,file='data.normAfterConfigMDPbiomePreprocess.RData')
tableSerie <- stateSerieTable(data.norm, stepVar, subjectId = subjectVar)
timeSerieHeatMap(tableSerie, "./", "statesSequence_allSamples.pdf")

In [ ]:
createTreeDir(dirdata,Perturbations)
options(max.print = 9999999)
# Build model and compute stability evaluation
mdpBiomeBase(goalDiversity=FALSE, utilityVar=goalVar) 

In [ ]:
###USER-INTERACTION###
# Compute generality evaluation
# It is mandatory to define rewardType ("preferGood", "avoidBad" or "proportional")
titledata=labelExp
mdpBiomeLoocv(goal_preference=cluster_preference,rewardType="avoidBad",goalVar=goalVar)
# It usually takes a long time to finish (10-30 minutes), mainly depending 
# on the number of subjects

In [ ]:
# ALTERNATIVE RUN
### Re-run MDPbiome, when the preprocess R data was saved before
setwd('../Src')
source("initMDPBiome.R")
dirdata <- "../Data/"
labelExp <- "community1"
setwd(dirdata)
load('data.normAfterConfigMDPbiomePreprocess.RData')
Perturbations <- c('pert')
stepVar <- "time"
subjectVar <- "subject"
cluster_preference <- goal_preference
createTreeDir(dirdata,Perturbations)
options(max.print = 9999999)
mdpBiomeBase(goalDiversity=FALSE, utilityVar=goalVar)
